In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import sys
sys.path.append('/home/simon/Research/lib/caffe.py3/python/')
import caffe
caffe.set_device(0)
caffe.set_mode_gpu()
import Caffe
import Resize
import Pad
import PIL
import Classification
import glob

import matplotlib
%matplotlib inline  
import matplotlib.pyplot as plt
import Dataset
import Evaluation
import ImageReader
import Crop
import MulticlassSVM
import Classificator
import sklearn
import Lambda
import subprocess
import Blob

In [ ]:
basedir = '/home/simon/Research/generic/results/2016-09-12_bilinear_pairwise_matching/dataset/'
#basedir = '/home/simon/Datasets/CUB_200_2011/'
d = Dataset.Dataset()
d.read_from_file('%s/imagelist.txt'%basedir, 'imagepaths','string')
d.read_from_file('%s/labels.txt'%basedir,'labels','int')
d.read_from_file('%s/tr_ID.txt'%basedir,'split_assignments','int')

In [ ]:
def trilinear(x):
    x = x.reshape((512,-1))
    return x.dot(x.T).dot(x).reshape((512,28,28))

In [ ]:
def bilinear(x):
    x = x.reshape((512,-1))
    return x.dot(x.T)

In [ ]:
from IAlgorithm import IAlgorithm
class FlipAugmentation(IAlgorithm):
    def _train(self, blob_generator):
        for blob in blob_generator:
            b = Blob.Blob()
            b.meta = blob.meta
            b.data = blob.data
            yield b
            
            b = Blob.Blob()
            b.meta = blob.meta
            b.data = np.fliplr(blob.data)
            yield b
            
    def _compute(self, blob_generator):
        for blob in blob_generator:
            yield blob

In [ ]:
c = Classification.Classification()
c.add_algorithm(ImageReader.ImageReader())
c.add_algorithm(Resize.Resize((448,), mode='resize_smaller_side'))
c.add_algorithm(Crop.Crop((448,448),'center'))
#c.add_algorithm(Resize.Resize((448,448), mode='stretch'))
#c.add_algorithm(FlipAugmentation())
#c.add_algorithm(Pad.Pad(227,227))
modeldir = '//home/simon/Data/caffe/vgg16/cub200_bilinear//'
#modeldir = '/home/simon/Research/generic/results/2016-07-01_cnn_imagenet_training/alexnet-visualization-nofc/alexnet_vis_nofc_imagenet/'
#modeldir = '/home/simon/Research/generic/results/2016-07-01_cnn_imagenet_training/alexnet-bn/alexnet_nofinegrained_allbn/'
subprocess.check_call(['touch',modeldir + '/deploy.prototxt'])
c.add_algorithm(Caffe.Caffe(modeldir + '/deploy.prototxt',
                           modeldir + '/model',
                           #mean = np.float32([0,0,0]),
                           outblob = 'bilinear_l2',
                           batchsize = 5))
#c.add_algorithm(Lambda.Lambda(lambda x: trilinear(x)))
#c.add_algorithm(Lambda.Lambda(lambda x: x.mean(axis=(1,2))))
#c.add_algorithm(Lambda.Lambda(lambda x: x/np.sqrt((x.ravel()**2).sum())))
c.add_algorithm(Classificator.Classificator(sklearn.svm.LinearSVC(C=1)))
#c.add_algorithm(Classificator.Classificator(sklearn.svm.SVC(C=1.0, kernel='linear', degree=2)))

In [ ]:
ev = Evaluation.Evaluation()
ev.fixed_split_eval(d, c)

In [ ]:
np.hstack(c.compute_all([im for im,s in zip(d.imagepaths,d.split_assignments) if s==1]))==np.array([l for l,s in zip(d.labels,d.split_assignments) if s==1])

### CUB200 2011
Bilinear l2 alexnet (0.59837763203313776, 0.60197659494092859)

Trilinear 8192 (0.58474283741801869, 0.58805479109517222)

Quadlinear 512 (0.52813255091473943, 0.53082733856751463) 8192 (0.53693476009665175, 0.54000831986618703)

In [ ]:
test_images = [im for im,split in zip(d.imagepaths,d.split_assignments) if split==0]
test_labels = [label for label,split in zip(d.labels,d.split_assignments) if split==0]

In [ ]:
predictions = c.compute_all(test_images)

In [ ]:
np.savetxt('/home/simon/Research/finegrained/results/2016-09-02_single_pose_dataset/cub200_2011/vgg16_bilinear_pred/results.txt',np.vstack(predictions).astype(int),fmt='%i')